In [ ]:
import importlib as il
il.invalidate_caches()
glob               =il.import_module('glob')
os                 =il.import_module('os')
sys                =il.import_module('sys')
sys.path.insert(0,'/notebook_dir/writable-workspace/cccs-utilities/data-tools')
sys.path.insert(0,'/notebook_dir/writable-workspace/cccs-utilities/misc-tools')
Thredds_extraction_functions  =il.import_module('Thredds_extraction_functions')
ECCC_IDF_reader               =il.import_module('ECCC_IDF_reader')
SSH_SCP_connector             =il.import_module('SSH-SCP-connector')
xr                 =il.import_module('xarray')
xci                =il.import_module('xclim.indices')

np                 =il.import_module('numpy')
pd                 =il.import_module('pandas')
from tqdm import tqdm
from xclim.ensembles import create_ensemble
from scp import SCPClient

In [ ]:
rcps=['rcp26','rcp45','rcp85']

# Load temperature data
ds=[]
for rcp in rcps:
    file_list=Thredds_extraction_functions.threddscall(rcps=rcp,
                                                       indices='tg_mean',
                                                       frequency='YS',
                                                       average=True)
    ds.append(create_ensemble([f.opendap_url() for f in file_list]))
temperature_data=xr.concat(ds,dim='rcp')
temperature_data=temperature_data.rename_dims({'realization':'model'})
temperature_data=temperature_data.rename_vars({'realization':'model'})

mod_time_as_int=temperature_data.coords['time'].values.astype('datetime64[Y]').astype(int)+1970

In [ ]:
n_rcps=len(temperature_data.coords['rcp'])
n_models=len(temperature_data.coords['model'])
n_time=len(temperature_data.coords['time'])
n_durations=9
n_return_periods=6
n_vars=2 #raw #s, and 95% confidence ranges

In [ ]:
# Load IDF data
IDF_files=glob.glob('IDF-files/IDF_v-3.20_2021_03_26_*/*.txt')
tmp=ECCC_IDF_reader.read_ECCC_IDF(IDF_files[0])
n_stations=len(IDF_files)
station_names=[ECCC_IDF_reader.read_ECCC_IDF(f)['location']['name'] for f in IDF_files]
station_ID=[ECCC_IDF_reader.read_ECCC_IDF(f)['location']['ID'] for f in IDF_files]
station_latitudes=[ECCC_IDF_reader.read_ECCC_IDF(f)['location']['latitude'] for f in IDF_files]
station_longitudes=[ECCC_IDF_reader.read_ECCC_IDF(f)['location']['longitude'] for f in IDF_files]

In [ ]:
#create new, empty xarray dataset to hold all results
IDF_da=xr.DataArray(np.empty((n_stations,n_durations,n_return_periods,n_rcps,n_models,n_time)),
                         dims=[     'station', 'duration', 'return_period', 'rcp', 'model', 'time',],
                         coords=dict(station=station_ID,
                                     duration=tmp['IDF_rates'].index.values,
                                     return_period=tmp['IDF_rates'].columns.values,
                                     rcp=rcps,
                                     models=temperature_data.coords['model'],
                                     time=temperature_data.coords['time'],
                                    ))

IDF_confidence_da=xr.DataArray(np.empty((n_stations,n_durations,n_return_periods)),
                         dims=[     'station', 'duration', 'return_period'],
                         coords=dict(station=station_ID,
                                     duration=tmp['IDF_rates'].index.values,
                                     return_period=tmp['IDF_rates'].columns.values,))

IDF_station_latitude_da=xr.DataArray(np.empty((n_stations)),
                         dims=[     'station'],
                         coords=dict(station=station_ID,))

IDF_station_longitude_da=xr.DataArray(np.empty((n_stations)),
                         dims=[     'station'],
                         coords=dict(station=station_ID,))

for n,f in enumerate(tqdm(IDF_files)):
    IDF_dict=ECCC_IDF_reader.read_ECCC_IDF(f)
    T=temperature_data['tg_mean'].sel(dict(lat=IDF_dict['location']['latitude'],
                                        lon=IDF_dict['location']['longitude']*-1.),
                                        method='nearest')

    #Get mid-point of IDF period of record
    base_time=int(np.mean((IDF_dict['period']['start_date'],
                           IDF_dict['period']['end_date'])))
    ibase_T=np.argmin(abs(mod_time_as_int-base_time-15))
    
    print(ibase_T)
    
    dT=T-T[:,:,ibase_T] #calculate dT - broadcast to all models, RCPs, times
    dT.attrs['units']='delta_degreeC'
    
    pr_baseline=xr.DataArray(IDF_dict['IDF_rates'],
                        coords=dict(duration=IDF_da.coords['duration'],
                                    return_period=IDF_da.coords['return_period']),
                        attrs={"units":"mm/hour"})
    pr_future=xci.clausius_clapeyron_scaled_precipitation(dT,pr_baseline)

    
    IDF_da.loc[IDF_dict['location']['ID'],:,:,:,:,:] = pr_future
    IDF_confidence_da.loc[IDF_dict['location']['ID'],:,:] = IDF_dict["IDF_rate_confidence"]
    
    IDF_station_latitude_da.loc[IDF_dict['location']['ID']] = IDF_dict['location']['latitude']
    IDF_station_longitude_da.loc[IDF_dict['location']['ID']] = IDF_dict['location']['longitude']
    #TODO: add station name variable accumulator



In [ ]:
IDF_df=xr.Dataset(dict(IDF_data=IDF_da,
                       IDF_confidence=IDF_confidence_da,
                       IDF_station_latitude=IDF_station_latitude_da,
                       IDF_station_longitude=IDF_station_longitude_da,))

In [ ]:
fname='national_IDF_projection_dataset.nc'

In [ ]:
IDF_df.to_netcdf(fname)

In [ ]:


ssh=SSH_SCP_connector.createSSHClient('ZZZ',
                                         22,
                                         'YYY',
                                         'XXX')

In [ ]:
scp = SCPClient(ssh.get_transport())
scp.put('XXX'+fname,'YYY')

(stdin, stdout, stderr)=ssh.exec_command('chmod a+x '+collab_server_data_path+'figs')
for line in stdout.readlines():
        print(line)

ssh.close()


In [ ]:
!ncdump -h national_IDF_projection_dataset.nc

In [ ]:
!wget ***REMOVED***~***REMOVED***/national_IDF_projection_dataset.nc